# ML Algorithms From Scratch - Cisco Coding Round Prep

## Top 4 ML Algorithms Asked in Interviews

Based on 2025 interview trends, these are the **ONLY 4 algorithms commonly asked**:
1. **Linear Regression**
2. **Logistic Regression** 
3. **K-Nearest Neighbors (k-NN)**
4. **K-Means Clustering**

**Key Rule:** Interviewers expect you to implement these **WITHOUT sklearn** - only NumPy allowed!

### Interview Examples from FAANG:
- **Uber**: "Write AUC from scratch using vanilla Python"
- **Google**: "Write K-Means using NumPy only"
- **Startup**: "Code Gradient Descent from scratch using NumPy and SciPy only"

---

In [ ]:
# Essential imports - ONLY these allowed in most interviews
import numpy as np
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

# For testing only (won't be available in interview)
from sklearn.datasets import make_regression, make_classification, make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

---
## Algorithm 1: Linear Regression

**Time:** 20-25 minutes  
**Difficulty:** Easy-Medium  
**Asked by:** Google, Meta, Amazon

Implement Linear Regression using:
1. Normal Equation (closed-form)
2. Gradient Descent (iterative)

### Mathematical Background

**Hypothesis:** $h_\theta(x) = \theta_0 + \theta_1 x_1 + ... + \theta_n x_n = \theta^T x$

**Cost Function (MSE):** $J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)})^2$

**Normal Equation:** $\theta = (X^T X)^{-1} X^T y$

**Gradient:** $\frac{\partial J}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)}$

---

### Method 1: Normal Equation (Closed-Form Solution)

In [ ]:
class LinearRegressionNormal:
    """
    Linear Regression using Normal Equation
    
    Time Complexity: O(n^3) due to matrix inversion
    Space Complexity: O(n^2)
    
    Pros: No hyperparameters, exact solution
    Cons: Slow for large n, doesn't work if X^T X is singular
    """
    
    def __init__(self):
        self.theta = None
    
    def fit(self, X, y):
        """
        Fit using Normal Equation: θ = (X^T X)^(-1) X^T y
        
        Args:
            X: (m, n) feature matrix
            y: (m,) target vector
        """
        # Add intercept term (column of ones)
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        
        # Normal equation: θ = (X^T X)^(-1) X^T y
        self.theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
        
        return self
    
    def predict(self, X):
        """
        Predict using θ^T x
        """
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b @ self.theta
    
    def score(self, X, y):
        """
        Calculate R^2 score
        """
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - (ss_res / ss_tot)

# Test
X, y = make_regression(n_samples=100, n_features=1, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegressionNormal()
model.fit(X_train, y_train)

print(f"Theta (weights): {model.theta}")
print(f"R^2 Score: {model.score(X_test, y_test):.4f}")

# Visualize
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.scatter(X_train, y_train, alpha=0.5, label='Training data')
plt.plot(X_train, model.predict(X_train), 'r-', label='Fitted line')
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.title('Linear Regression - Normal Equation')

plt.subplot(1, 2, 2)
y_pred = model.predict(X_test)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('Predictions vs True Values')
plt.tight_layout()
plt.show()

### Method 2: Gradient Descent (Iterative Solution)

In [ ]:
class LinearRegressionGD:
    """
    Linear Regression using Gradient Descent
    
    Time Complexity: O(m * n * iterations)
    Space Complexity: O(n)
    
    Pros: Works for large datasets, can use mini-batch/stochastic variants
    Cons: Requires hyperparameter tuning (learning rate, iterations)
    """
    
    def __init__(self, learning_rate=0.01, n_iterations=1000, tolerance=1e-6):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.tolerance = tolerance
        self.theta = None
        self.cost_history = []
    
    def fit(self, X, y, verbose=False):
        """
        Fit using Gradient Descent
        
        Update rule: θ = θ - α * (1/m) * X^T * (X*θ - y)
        """
        # Add intercept
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        m, n = X_b.shape
        
        # Initialize theta randomly
        self.theta = np.random.randn(n)
        
        # Gradient descent
        for iteration in range(self.n_iterations):
            # Predictions
            predictions = X_b @ self.theta
            
            # Errors
            errors = predictions - y
            
            # Compute cost (MSE)
            cost = (1/(2*m)) * np.sum(errors ** 2)
            self.cost_history.append(cost)
            
            # Compute gradient: (1/m) * X^T * errors
            gradient = (1/m) * X_b.T @ errors
            
            # Update parameters
            self.theta = self.theta - self.learning_rate * gradient
            
            # Early stopping
            if iteration > 0 and abs(self.cost_history[-1] - self.cost_history[-2]) < self.tolerance:
                if verbose:
                    print(f"Converged at iteration {iteration}")
                break
            
            if verbose and iteration % 100 == 0:
                print(f"Iteration {iteration}: Cost = {cost:.4f}")
        
        return self
    
    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b @ self.theta
    
    def score(self, X, y):
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - (ss_res / ss_tot)

# Test
model_gd = LinearRegressionGD(learning_rate=0.01, n_iterations=1000)
model_gd.fit(X_train, y_train, verbose=True)

print(f"\nTheta (weights): {model_gd.theta}")
print(f"R^2 Score: {model_gd.score(X_test, y_test):.4f}")

# Plot cost history
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(model_gd.cost_history)
plt.xlabel('Iteration')
plt.ylabel('Cost (MSE)')
plt.title('Cost Function Convergence')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(X_test, y_test, alpha=0.5, label='Test data')
plt.plot(X_test, model_gd.predict(X_test), 'r-', label='Predictions')
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.title('Test Set Predictions')
plt.tight_layout()
plt.show()

### Interview Discussion Points

**Q: When to use Normal Equation vs Gradient Descent?**
- **Normal Equation:** n < 10,000 features, need exact solution, no hyperparameter tuning
- **Gradient Descent:** Large datasets, n > 10,000, can use mini-batch/SGD, more scalable

**Q: How to improve Gradient Descent?**
1. **Feature Scaling:** Normalize features to [0, 1] or standardize to mean=0, std=1
2. **Learning Rate Scheduling:** Decrease learning rate over time
3. **Mini-batch/SGD:** Use subsets of data for faster updates
4. **Momentum:** Add momentum to escape local minima

**Q: What if X^T X is singular (non-invertible)?**
- Use **Regularization** (Ridge/Lasso)
- Remove redundant/correlated features
- Use SVD decomposition instead of inverse
- Switch to Gradient Descent

---

## Algorithm 2: Logistic Regression

**Time:** 25-30 minutes  
**Difficulty:** Medium  
**Asked by:** Facebook, Google, Uber

Binary classification using logistic regression.

### Mathematical Background

**Hypothesis:** $h_\theta(x) = \sigma(\theta^T x) = \frac{1}{1 + e^{-\theta^T x}}$

**Cost Function (Cross-Entropy):**  
$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} [y^{(i)} \log(h_\theta(x^{(i)})) + (1-y^{(i)}) \log(1-h_\theta(x^{(i)}))]$

**Gradient:**  
$\frac{\partial J}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)}$

---

In [ ]:
class LogisticRegression:
    """
    Logistic Regression for Binary Classification
    
    Uses Gradient Descent with Cross-Entropy loss
    """
    
    def __init__(self, learning_rate=0.01, n_iterations=1000, tolerance=1e-6):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.tolerance = tolerance
        self.theta = None
        self.cost_history = []
    
    def sigmoid(self, z):
        """
        Sigmoid function with numerical stability
        
        σ(z) = 1 / (1 + e^(-z))
        """
        # Clip to prevent overflow
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))
    
    def compute_cost(self, X, y, theta):
        """
        Cross-Entropy Cost Function
        
        J(θ) = -1/m * Σ[y*log(h) + (1-y)*log(1-h)]
        """
        m = len(y)
        h = self.sigmoid(X @ theta)
        
        # Add small epsilon to prevent log(0)
        epsilon = 1e-10
        cost = -1/m * np.sum(
            y * np.log(h + epsilon) + (1 - y) * np.log(1 - h + epsilon)
        )
        
        return cost
    
    def fit(self, X, y, verbose=False):
        """
        Fit using Gradient Descent
        """
        # Add intercept
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        m, n = X_b.shape
        
        # Initialize theta
        self.theta = np.zeros(n)
        
        # Gradient descent
        for iteration in range(self.n_iterations):
            # Predictions
            h = self.sigmoid(X_b @ self.theta)
            
            # Errors
            errors = h - y
            
            # Compute cost
            cost = self.compute_cost(X_b, y, self.theta)
            self.cost_history.append(cost)
            
            # Compute gradient
            gradient = (1/m) * X_b.T @ errors
            
            # Update parameters
            self.theta = self.theta - self.learning_rate * gradient
            
            # Early stopping
            if iteration > 0 and abs(self.cost_history[-1] - self.cost_history[-2]) < self.tolerance:
                if verbose:
                    print(f"Converged at iteration {iteration}")
                break
            
            if verbose and iteration % 100 == 0:
                print(f"Iteration {iteration}: Cost = {cost:.4f}")
        
        return self
    
    def predict_proba(self, X):
        """
        Return probabilities
        """
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return self.sigmoid(X_b @ self.theta)
    
    def predict(self, X, threshold=0.5):
        """
        Return binary predictions
        """
        return (self.predict_proba(X) >= threshold).astype(int)
    
    def score(self, X, y):
        """
        Calculate accuracy
        """
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

# Test
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, 
                          n_informative=2, random_state=42, n_clusters_per_class=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_lr = LogisticRegression(learning_rate=0.1, n_iterations=1000)
model_lr.fit(X_train, y_train, verbose=True)

print(f"\nTest Accuracy: {model_lr.score(X_test, y_test):.4f}")

# Visualize decision boundary
plt.figure(figsize=(12, 4))

# Plot 1: Cost history
plt.subplot(1, 3, 1)
plt.plot(model_lr.cost_history)
plt.xlabel('Iteration')
plt.ylabel('Cost (Cross-Entropy)')
plt.title('Cost Function Convergence')
plt.grid(True)

# Plot 2: Decision boundary
plt.subplot(1, 3, 2)
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))
Z = model_lr.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, edgecolors='k', alpha=0.7)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Decision Boundary')

# Plot 3: Probability distribution
plt.subplot(1, 3, 3)
probs = model_lr.predict_proba(X_test)
plt.hist(probs[y_test == 0], bins=20, alpha=0.5, label='Class 0')
plt.hist(probs[y_test == 1], bins=20, alpha=0.5, label='Class 1')
plt.xlabel('Predicted Probability')
plt.ylabel('Count')
plt.title('Probability Distribution')
plt.legend()
plt.tight_layout()
plt.show()

### Interview Discussion Points

**Q: Why can't we use MSE for logistic regression?**
- MSE creates non-convex cost function with logistic hypothesis
- Gradient descent might get stuck in local minima
- Cross-entropy is convex and better suited for classification

**Q: How to handle class imbalance?**
1. **Weighted loss:** Give more weight to minority class
2. **Threshold tuning:** Adjust decision threshold based on precision/recall needs
3. **SMOTE:** Oversample minority class
4. **Undersampling:** Downsample majority class

**Q: Logistic Regression vs Neural Network?**
- LR: Linear decision boundary, interpretable, fast, good for linearly separable data
- NN: Non-linear, flexible, slower, better for complex patterns

---

## Algorithm 3: K-Nearest Neighbors (k-NN)

**Time:** 20-25 minutes  
**Difficulty:** Easy-Medium  
**Asked by:** Amazon, Microsoft

Instance-based learning algorithm for classification and regression.

### Algorithm
1. Store all training data
2. For new point, find k nearest neighbors
3. **Classification:** Majority vote
4. **Regression:** Average of k neighbors

---

In [ ]:
class KNNClassifier:
    """
    K-Nearest Neighbors Classifier
    
    Time Complexity:
    - Training: O(1) - just store data
    - Prediction: O(n * d) for each query where n=samples, d=features
    
    Space Complexity: O(n * d)
    """
    
    def __init__(self, k=3, distance_metric='euclidean'):
        """
        Args:
            k: Number of neighbors
            distance_metric: 'euclidean' or 'manhattan'
        """
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
    
    def compute_distance(self, x1, x2):
        """
        Compute distance between two points
        """
        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2, axis=1))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(x1 - x2), axis=1)
        else:
            raise ValueError(f"Unknown distance metric: {self.distance_metric}")
    
    def fit(self, X, y):
        """
        Store training data (lazy learning)
        """
        self.X_train = X
        self.y_train = y
        return self
    
    def predict_single(self, x):
        """
        Predict label for single instance
        """
        # Compute distances to all training points
        distances = self.compute_distance(self.X_train, x)
        
        # Get indices of k nearest neighbors
        k_indices = np.argsort(distances)[:self.k]
        
        # Get labels of k nearest neighbors
        k_nearest_labels = self.y_train[k_indices]
        
        # Majority vote
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]
    
    def predict(self, X):
        """
        Predict labels for multiple instances
        """
        return np.array([self.predict_single(x) for x in X])
    
    def score(self, X, y):
        """
        Calculate accuracy
        """
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

# Test
X, y = make_classification(n_samples=500, n_features=2, n_redundant=0,
                          n_informative=2, random_state=42, n_clusters_per_class=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Try different k values
k_values = [1, 3, 5, 10, 20]
scores = []

for k in k_values:
    knn = KNNClassifier(k=k)
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    scores.append(score)
    print(f"k={k}: Accuracy = {score:.4f}")

# Visualize
plt.figure(figsize=(12, 4))

# Plot 1: k vs Accuracy
plt.subplot(1, 3, 1)
plt.plot(k_values, scores, 'bo-')
plt.xlabel('k (number of neighbors)')
plt.ylabel('Accuracy')
plt.title('k vs Accuracy')
plt.grid(True)

# Plot 2: Decision boundary for k=3
plt.subplot(1, 3, 2)
knn_best = KNNClassifier(k=3)
knn_best.fit(X_train, y_train)
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 50),
                     np.linspace(y_min, y_max, 50))
Z = knn_best.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, edgecolors='k', alpha=0.7)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Decision Boundary (k=3)')

# Plot 3: Training data
plt.subplot(1, 3, 3)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', alpha=0.7)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Training Data')
plt.tight_layout()
plt.show()

### Optimized k-NN using Vectorization

In [ ]:
class KNNClassifierVectorized:
    """
    Vectorized k-NN for better performance
    
    Uses matrix operations instead of loops
    """
    
    def __init__(self, k=3):
        self.k = k
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        return self
    
    def predict(self, X):
        """
        Vectorized prediction using broadcasting
        
        Distance matrix: (n_test, n_train)
        """
        # Compute pairwise distances using broadcasting
        # ||x - y||^2 = ||x||^2 + ||y||^2 - 2*x*y
        X_train_sq = np.sum(self.X_train ** 2, axis=1)
        X_test_sq = np.sum(X ** 2, axis=1)[:, np.newaxis]
        cross_term = -2 * X @ self.X_train.T
        
        distances = np.sqrt(X_test_sq + X_train_sq + cross_term)
        
        # Get k nearest neighbors for each test point
        k_indices = np.argpartition(distances, self.k, axis=1)[:, :self.k]
        
        # Get labels of k nearest neighbors
        k_nearest_labels = self.y_train[k_indices]
        
        # Majority vote using bincount (faster than Counter for large datasets)
        predictions = np.array([
            np.bincount(labels).argmax() 
            for labels in k_nearest_labels
        ])
        
        return predictions
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

# Compare performance
import time

X_large, y_large = make_classification(n_samples=2000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_large, y_large, test_size=0.2)

# Original k-NN
start = time.time()
knn_original = KNNClassifier(k=5)
knn_original.fit(X_train, y_train)
score_original = knn_original.score(X_test, y_test)
time_original = time.time() - start

# Vectorized k-NN
start = time.time()
knn_vectorized = KNNClassifierVectorized(k=5)
knn_vectorized.fit(X_train, y_train)
score_vectorized = knn_vectorized.score(X_test, y_test)
time_vectorized = time.time() - start

print("Performance Comparison:")
print(f"Original k-NN: {time_original:.4f}s, Accuracy: {score_original:.4f}")
print(f"Vectorized k-NN: {time_vectorized:.4f}s, Accuracy: {score_vectorized:.4f}")
print(f"Speedup: {time_original / time_vectorized:.2f}x faster")

### Interview Discussion Points

**Q: How to choose k?**
- Small k: More flexible, sensitive to noise, higher variance
- Large k: Smoother boundary, less sensitive to noise, higher bias
- Use cross-validation to find optimal k
- Rule of thumb: k = sqrt(n)

**Q: Pros and Cons of k-NN?**

**Pros:**
- Simple to understand and implement
- No training phase (lazy learning)
- Naturally handles multi-class
- Can model complex decision boundaries

**Cons:**
- Slow prediction (O(n) for each query)
- Memory intensive (stores all training data)
- Sensitive to feature scaling
- Curse of dimensionality (doesn't work well in high dimensions)

**Q: How to make k-NN faster?**
1. **KD-Trees/Ball Trees:** O(log n) search instead of O(n)
2. **Approximate NN:** LSH (Locality Sensitive Hashing)
3. **Dimensionality Reduction:** PCA before k-NN
4. **Feature Selection:** Remove irrelevant features

---

## Algorithm 4: K-Means Clustering

**Time:** 30-35 minutes  
**Difficulty:** Medium  
**Asked by:** Google, Amazon, Facebook

Unsupervised learning algorithm for clustering.

### Algorithm
1. Initialize k centroids randomly
2. **Assignment Step:** Assign each point to nearest centroid
3. **Update Step:** Recompute centroids as mean of assigned points
4. Repeat 2-3 until convergence

**Objective:** Minimize within-cluster sum of squares (WCSS)

$J = \sum_{i=1}^{k} \sum_{x \in C_i} ||x - \mu_i||^2$

---

In [ ]:
class KMeans:
    """
    K-Means Clustering
    
    Time Complexity: O(n * k * d * iterations)
    Space Complexity: O(n + k)
    
    where n=samples, k=clusters, d=features
    """
    
    def __init__(self, n_clusters=3, max_iters=100, random_state=42, tolerance=1e-4):
        self.n_clusters = n_clusters
        self.max_iters = max_iters
        self.random_state = random_state
        self.tolerance = tolerance
        self.centroids = None
        self.labels = None
        self.inertia_ = None
        self.n_iter_ = 0
    
    def initialize_centroids(self, X):
        """
        Initialize centroids randomly from data points
        """
        np.random.seed(self.random_state)
        random_indices = np.random.choice(X.shape[0], self.n_clusters, replace=False)
        return X[random_indices]
    
    def compute_distances(self, X, centroids):
        """
        Compute distances from each point to each centroid
        
        Returns: (n_samples, n_clusters) matrix
        """
        distances = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            distances[:, k] = np.linalg.norm(X - centroids[k], axis=1)
        return distances
    
    def assign_clusters(self, X, centroids):
        """
        Assign each point to nearest centroid
        """
        distances = self.compute_distances(X, centroids)
        return np.argmin(distances, axis=1)
    
    def update_centroids(self, X, labels):
        """
        Update centroids as mean of assigned points
        """
        centroids = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(self.n_clusters):
            cluster_points = X[labels == k]
            if len(cluster_points) > 0:
                centroids[k] = cluster_points.mean(axis=0)
            else:
                # Handle empty cluster - reinitialize randomly
                centroids[k] = X[np.random.choice(X.shape[0])]
        return centroids
    
    def compute_inertia(self, X, labels, centroids):
        """
        Compute within-cluster sum of squares
        """
        inertia = 0
        for k in range(self.n_clusters):
            cluster_points = X[labels == k]
            if len(cluster_points) > 0:
                inertia += np.sum((cluster_points - centroids[k]) ** 2)
        return inertia
    
    def fit(self, X, verbose=False):
        """
        Fit K-Means clustering
        """
        # Initialize centroids
        self.centroids = self.initialize_centroids(X)
        
        for iteration in range(self.max_iters):
            # Assignment step
            old_centroids = self.centroids.copy()
            self.labels = self.assign_clusters(X, self.centroids)
            
            # Update step
            self.centroids = self.update_centroids(X, self.labels)
            
            # Check convergence
            centroid_shift = np.linalg.norm(self.centroids - old_centroids)
            
            if verbose:
                inertia = self.compute_inertia(X, self.labels, self.centroids)
                print(f"Iteration {iteration}: Inertia = {inertia:.4f}, Shift = {centroid_shift:.6f}")
            
            if centroid_shift < self.tolerance:
                if verbose:
                    print(f"Converged at iteration {iteration}")
                break
        
        self.n_iter_ = iteration + 1
        self.inertia_ = self.compute_inertia(X, self.labels, self.centroids)
        
        return self
    
    def predict(self, X):
        """
        Predict cluster labels for new data
        """
        return self.assign_clusters(X, self.centroids)
    
    def fit_predict(self, X):
        """
        Fit and return cluster labels
        """
        self.fit(X)
        return self.labels

# Test
X, y_true = make_blobs(n_samples=500, centers=4, n_features=2, random_state=42)

kmeans = KMeans(n_clusters=4, max_iters=100, random_state=42)
kmeans.fit(X, verbose=True)

print(f"\nFinal Inertia: {kmeans.inertia_:.4f}")
print(f"Number of iterations: {kmeans.n_iter_}")
print(f"Centroids:\n{kmeans.centroids}")

# Visualize
plt.figure(figsize=(15, 5))

# Plot 1: True labels
plt.subplot(1, 3, 1)
plt.scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', alpha=0.6, edgecolors='k')
plt.title('True Labels')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# Plot 2: Predicted clusters
plt.subplot(1, 3, 2)
plt.scatter(X[:, 0], X[:, 1], c=kmeans.labels, cmap='viridis', alpha=0.6, edgecolors='k')
plt.scatter(kmeans.centroids[:, 0], kmeans.centroids[:, 1], 
           c='red', marker='X', s=200, edgecolors='black', linewidth=2, label='Centroids')
plt.title('K-Means Clustering')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()

# Plot 3: Elbow method
plt.subplot(1, 3, 3)
inertias = []
K_range = range(1, 10)
for k in K_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X)
    inertias.append(km.inertia_)

plt.plot(K_range, inertias, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia (WCSS)')
plt.title('Elbow Method')
plt.grid(True)
plt.tight_layout()
plt.show()

### K-Means++: Better Initialization

In [ ]:
class KMeansPlusPlus(KMeans):
    """
    K-Means with K-Means++ initialization
    
    Better centroid initialization for faster convergence
    """
    
    def initialize_centroids(self, X):
        """
        K-Means++ initialization
        
        1. Choose first centroid randomly
        2. For each subsequent centroid:
           - Compute D(x) = distance to nearest centroid
           - Choose new centroid with probability proportional to D(x)^2
        """
        np.random.seed(self.random_state)
        
        # Choose first centroid randomly
        centroids = [X[np.random.choice(X.shape[0])]]
        
        # Choose remaining k-1 centroids
        for _ in range(1, self.n_clusters):
            # Compute distances to nearest centroid
            distances = np.array([
                min(np.linalg.norm(x - c) ** 2 for c in centroids)
                for x in X
            ])
            
            # Choose next centroid with probability proportional to distance^2
            probabilities = distances / distances.sum()
            next_centroid_idx = np.random.choice(X.shape[0], p=probabilities)
            centroids.append(X[next_centroid_idx])
        
        return np.array(centroids)

# Compare K-Means vs K-Means++
X, y_true = make_blobs(n_samples=1000, centers=5, n_features=2, random_state=42)

# Regular K-Means
kmeans_regular = KMeans(n_clusters=5, random_state=42)
kmeans_regular.fit(X)

# K-Means++
kmeans_pp = KMeansPlusPlus(n_clusters=5, random_state=42)
kmeans_pp.fit(X)

print("Comparison:")
print(f"Regular K-Means: {kmeans_regular.n_iter_} iterations, Inertia: {kmeans_regular.inertia_:.2f}")
print(f"K-Means++: {kmeans_pp.n_iter_} iterations, Inertia: {kmeans_pp.inertia_:.2f}")

### Interview Discussion Points

**Q: Limitations of K-Means?**
1. **Must specify k:** Need to know number of clusters beforehand
2. **Sensitive to initialization:** Different random starts give different results
3. **Assumes spherical clusters:** Doesn't work well for elongated or irregular shapes
4. **Sensitive to outliers:** Outliers can skew centroids
5. **Euclidean distance:** Assumes all features equally important

**Q: How to choose k?**
1. **Elbow Method:** Plot inertia vs k, look for "elbow"
2. **Silhouette Score:** Measure how similar point is to its cluster vs other clusters
3. **Domain Knowledge:** Use business understanding
4. **Cross-Validation:** For downstream task

**Q: K-Means vs Hierarchical Clustering?**
- **K-Means:** Faster O(nkd), requires k, produces flat clusters
- **Hierarchical:** Slower O(n³), no need for k, produces dendrogram, deterministic

**Q: How to handle categorical features?**
- K-Modes (uses mode instead of mean)
- One-hot encoding + K-Means
- Use appropriate distance metric (Hamming distance)

---

## Bonus: Commonly Asked Supporting Functions

These are frequently asked as part of main algorithms.

---

### 1. Train-Test Split

In [ ]:
def train_test_split_scratch(X, y, test_size=0.2, random_state=None):
    """
    Split data into train and test sets
    
    Args:
        X: Features (n_samples, n_features)
        y: Labels (n_samples,)
        test_size: Fraction for test set
        random_state: Seed for reproducibility
    
    Returns:
        X_train, X_test, y_train, y_test
    """
    if random_state is not None:
        np.random.seed(random_state)
    
    n_samples = X.shape[0]
    n_test = int(n_samples * test_size)
    
    # Random shuffle indices
    indices = np.random.permutation(n_samples)
    
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

# Test
X = np.arange(100).reshape(20, 5)
y = np.arange(20)
X_train, X_test, y_train, y_test = train_test_split_scratch(X, y, test_size=0.2, random_state=42)
print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")

### 2. Feature Scaling

In [ ]:
class StandardScaler:
    """
    Standardize features: (x - mean) / std
    """
    
    def __init__(self):
        self.mean = None
        self.std = None
    
    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)
        return self
    
    def transform(self, X):
        return (X - self.mean) / (self.std + 1e-8)
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)

class MinMaxScaler:
    """
    Scale features to [0, 1]: (x - min) / (max - min)
    """
    
    def __init__(self):
        self.min = None
        self.max = None
    
    def fit(self, X):
        self.min = np.min(X, axis=0)
        self.max = np.max(X, axis=0)
        return self
    
    def transform(self, X):
        return (X - self.min) / (self.max - self.min + 1e-8)
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)

# Test
X = np.random.randn(100, 3) * 10 + 50

scaler_std = StandardScaler()
X_std = scaler_std.fit_transform(X)
print(f"Standardized - Mean: {X_std.mean(axis=0)}, Std: {X_std.std(axis=0)}")

scaler_minmax = MinMaxScaler()
X_minmax = scaler_minmax.fit_transform(X)
print(f"MinMax - Min: {X_minmax.min(axis=0)}, Max: {X_minmax.max(axis=0)}")

### 3. Cross-Validation

In [ ]:
def k_fold_cross_validation(X, y, model_class, k=5, **model_params):
    """
    K-Fold Cross-Validation
    
    Args:
        X: Features
        y: Labels
        model_class: Model class to instantiate
        k: Number of folds
        **model_params: Parameters for model
    
    Returns:
        List of scores for each fold
    """
    n_samples = X.shape[0]
    fold_size = n_samples // k
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    scores = []
    
    for fold in range(k):
        # Create train/val split
        val_start = fold * fold_size
        val_end = (fold + 1) * fold_size if fold < k - 1 else n_samples
        
        val_indices = indices[val_start:val_end]
        train_indices = np.concatenate([indices[:val_start], indices[val_end:]])
        
        X_train, X_val = X[train_indices], X[val_indices]
        y_train, y_val = y[train_indices], y[val_indices]
        
        # Train and evaluate
        model = model_class(**model_params)
        model.fit(X_train, y_train)
        score = model.score(X_val, y_val)
        scores.append(score)
        
        print(f"Fold {fold + 1}: Score = {score:.4f}")
    
    print(f"\nMean Score: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")
    return scores

# Test with Logistic Regression
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
scores = k_fold_cross_validation(X, y, LogisticRegression, k=5, 
                                 learning_rate=0.1, n_iterations=500)

---

## Interview Checklist

### Before the Interview
- [ ] Review all 4 core algorithms (Linear Regression, Logistic Regression, k-NN, K-Means)
- [ ] Practice implementing without sklearn
- [ ] Understand time/space complexity
- [ ] Know when to use each algorithm
- [ ] Practice explaining trade-offs

### During the Interview
- [ ] Clarify requirements (classification/regression, dataset size, constraints)
- [ ] Ask about allowed libraries (NumPy? SciPy? sklearn?)
- [ ] Discuss approach before coding
- [ ] Write clean, modular code
- [ ] Test with simple example
- [ ] Discuss complexity and optimizations

### Common Follow-up Questions
1. How would you optimize for large datasets?
2. What if data doesn't fit in memory?
3. How to handle missing values?
4. What about categorical features?
5. How to evaluate model performance?
6. What if features have different scales?

---

## Summary

You've now implemented the **TOP 4** ML algorithms asked in interviews:

1. ✅ **Linear Regression** - Normal Equation & Gradient Descent
2. ✅ **Logistic Regression** - Binary classification with cross-entropy
3. ✅ **K-Nearest Neighbors** - Instance-based learning with optimizations
4. ✅ **K-Means Clustering** - Unsupervised learning with K-Means++

**Plus supporting functions:**
- Train-test split
- Feature scaling (StandardScaler, MinMaxScaler)
- K-Fold cross-validation

**You're ready for ML coding interviews at Google, Facebook, Amazon, Microsoft, and Cisco!**

Good luck! 🚀
</cell>